In [58]:
import numpy as np
import pandas as pd
import os
import datetime
from dateutil.relativedelta import relativedelta
from binance.client import Client
from os.path import exists

In [59]:
client = Client()
exchange_info = client.get_exchange_info()
symbols = []
for symbol in exchange_info["symbols"]:
    symbols.append(symbol["symbol"])

In [60]:
intervals = ["1M", "1w", "3d", "1d", "12h", "8h", "6h", "4h", "2h", "1h", "30m", "15m", "5m", "3m", "1m"]
now = datetime.datetime.now()

In [61]:
for symbol in symbols:
    if (not exists(symbol)):
        os.mkdir(symbol)
        for interval in intervals:
            if interval == "1M":
                os.mkdir(symbol + "/" + "1Month")
            else:
                os.mkdir(symbol + "/" + interval)
    print(symbol)
    print(">> ", end = "")
    for interval in intervals:
        print(interval, end=" ")
        startDate = datetime.datetime(2016,1,1,0,0)
        if interval == "1m" or interval == "3m" or interval == "5m" or interval == "15m" or interval == "30m":
            endDate = startDate + relativedelta(months=3)
        elif interval == "1h" or interval == "2h" or interval == "4h" or interval == "6h":
            endDate = startDate + relativedelta(months=6)
        elif interval == "8h" or interval == "12h" or interval == "1d" or interval == "3d":
            endDate = startDate + relativedelta(months=12)
        elif interval == "1w" or interval == "1M": 
            endDate = startDate + relativedelta(years=3)
        endDate -= datetime.timedelta(seconds=1)

        start_str = str(startDate).split()[0]
        end_str = str(endDate).split()[0]
        while endDate < now:
            if (interval == "1m"):
                startDate = datetime.datetime(2016,1,1,0,0)
                endDate = startDate + relativedelta(months=3)
                endDate -= datetime.timedelta(seconds=1)
                start_str = str(startDate).split()[0]
                end_str = str(endDate).split()[0]


            bars = client.get_historical_klines(symbol=symbol,interval=interval,start_str=start_str, end_str=end_str)
            test_df = pd.DataFrame(bars[:],columns=["timestamp","open","high","low","close","volume", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"])
            test_df["date"]=pd.to_datetime(test_df["timestamp"],unit="ms").astype(str)
            # test_df = test_df.drop(["timestamp", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"], axis=1)
            test_df["open"] = pd.to_numeric(test_df["open"])
            test_df["high"] = pd.to_numeric(test_df["high"])
            test_df["low"] = pd.to_numeric(test_df["low"])
            test_df["close"] = pd.to_numeric(test_df["close"])
            test_df["volume"] = pd.to_numeric(test_df["volume"])

            if interval == "1M" and len(test_df) > 0:
                test_df.to_csv(f'{symbol}/1Month/from_{start_str}_to_{end_str}.csv')
            elif(len(test_df) > 0):
                test_df.to_csv(f'{symbol}/{interval}/from_{start_str}_to_{end_str}.csv')

            endDate += datetime.timedelta(seconds=1)
            startDate = endDate
            if interval == "1m" or interval == "3m" or interval == "5m" or interval == "15m" or interval == "30m":
                endDate = startDate + relativedelta(months=3)
            elif interval == "1h" or interval == "2h" or interval == "4h" or interval == "6h":
                endDate = startDate + relativedelta(months=6)
            elif interval == "8h" or interval == "12h" or interval == "1d" or interval == "3d":
                endDate = startDate + relativedelta(months=12)
            elif interval == "1w" or interval == "1M": 
                endDate = startDate + relativedelta(months=36)
            endDate -= datetime.timedelta(seconds=1)

            start_str = str(startDate).split()[0]
            end_str = str(endDate).split()[0]
    print()

ETHBTC
>> 1M 1w 3d 1d 12h 8h 6h 4h 2h 1h 30m 15m 5m 3m 